In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
import requests 
from IPython.core.display import display, HTML
from IPython.display import clear_output
import urllib
from urllib.request import urlopen
from urllib.parse import urlparse, urljoin, urldefrag
from collections import Counter
from bs4 import BeautifulSoup  
import hashlib

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'


with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url[:-1] for url in urls]

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_20509/2970182421.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [130]:
## utils functions
from openai import OpenAI
head_prompt = """I'll give you an HTML element that's extracted from the web page of a company. I want you to estimate the likelihood """ +\
"""that the hyperlink in the HTML element is likely to lead to a page that's named something like "About the Team" or "About Us" or "Our Team" or """+\
"""it leads to a Bio page of any employees in the company or to a location or branch of the company which could be a state or city, """+\
"""where there might be a list of employees"""+\
"""and I can subsequently get information about the employees' names, """ +\
"""phone numbers, emails etc. \nProvide your estimate as a single integer number between 0 and 100, where 0 means very unlikely, """ +\
"""and 100 means absolutely certain. Do not include any other text or explanation in your response. \n""" +\
"""Here's the HTML tag: """
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

client = OpenAI()

def call_gpt(atag, href, bios):
    prompt = head_prompt + str(atag)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers only and only in integer numeric values"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.01,
        logprobs=False
    ) 
    content = completion.choices[0].message.dict()['content']
    estimate = int(float(content))
    print(f'estimate:{str(estimate):4s} {href} ')
    if estimate >= 40:
        bios.append({"URL":href, "estimate":estimate})

    return estimate


def get_href(root_url, atag):
    href = atag.get("href")
    if not urlparse(href).hostname:
        href = urljoin(root_url, href)
    if urlparse(href).hostname is not None and \
        not urlparse(href).hostname.endswith(urlparse(root_url).hostname) and not urlparse(root_url).hostname.endswith(urlparse(href).hostname):
        print(f'        hostname mismatch: {urlparse(href).hostname} != {urlparse(root_url).hostname}')
        return None
    href = urldefrag(href).url
    return href
    
    
def hashit(text):
    return hashlib.md5(text.encode()).hexdigest()    
    
    
def get_atags(href):
    try:
        response = requests.get(href, headers=headers, allow_redirects=True, timeout=8)
        soup = BeautifulSoup(response.content, 'html.parser')
        atags = soup.find_all("a") 
        if not response.ok:
            print(f'    *** RESPONSE NOT OK!! *** status_code: {response.status_code} ')
        return response, atags
    except Exception:
        print(f'    Exception at href: {href} {Exception}')
        return None, None


In [133]:
## BFS
from collections import deque
start = time.time()
bios = []
seen = set()
model = "gpt-4-turbo"
maxdepth = 2

root_url = 'https://brokers.interexo.com/search?country%5B0%5D=United%20States'

queue = deque([(BeautifulSoup(f'<a href="{root_url}"/></a>', 'html.parser'), 0)]) 
seen.add(root_url)
prev_depth = -1
while queue and queue[0][1] <= maxdepth: 
    atag, depth = queue.popleft() 
    if prev_depth != depth:
        print(f'    ==> new depth:{depth}  len(queue):{len(queue)+1}')
        prev_depth = depth
    href = get_href(root_url, atag)
    if href is None:
        continue
    print(f'    depth:{depth} ', end=' ')
    #estimate = call_gpt(atag, href, bios)
    
    # ------------------------
    bios.append({"URL":href, "estimate":estimate}) 
    estimate = 100
    
    
    # ------------------------

    response, atags = get_atags(href)
    if atags is None or depth == maxdepth:
        continue
    for atag in atags:
        href = atag.get("href")
        if not urlparse(href).hostname:
            href = urljoin(root_url, href) 
        if urlparse(href).hostname is not None and \
            not urlparse(href).hostname.endswith(urlparse(root_url).hostname) and not urlparse(root_url).hostname.endswith(urlparse(href).hostname):
            continue
        if href not in seen:
            queue.append((atag, depth+1))
            seen.add(href)

print(f'    len(bios) {len(bios)} {print_runtime(start, False)}')

# with open ('bios.pkl', 'wb') as f:
#     pickle.dump(bios, f)

    ==> new depth:0  len(queue):1
    depth:0      ==> new depth:1  len(queue):72
    depth:1      depth:1      depth:1      depth:1      depth:1      depth:1      depth:1      depth:1      depth:1  

KeyboardInterrupt: 

In [132]:
bios

[{'URL': 'https://brokers.interexo.com/search?country%5B0%5D=United%20States',
  'estimate': 0},
 {'URL': 'https://brokers.interexo.com/', 'estimate': 100},
 {'URL': 'https://brokers.interexo.com/search', 'estimate': 100},
 {'URL': 'https://brokers.interexo.com/united-states_new-york/company/bridge-brokers/aj-caro-8e1',
  'estimate': 100},
 {'URL': 'https://brokers.interexo.com/united-states_new-york/company/bridge-brokers',
  'estimate': 100},
 {'URL': 'https://brokers.interexo.com/united-states_michigan/company/business-real-estate-network/ajohn-richwine-d4a',
  'estimate': 100},
 {'URL': 'https://brokers.interexo.com/united-states_michigan/company/business-real-estate-network',
  'estimate': 100},
 {'URL': 'https://brokers.interexo.com/united-states_south-carolina/company/industrypro/at-zirkle-b2d',
  'estimate': 100},
 {'URL': 'https://brokers.interexo.com/united-states_utah/company/industrypro',
  'estimate': 100},
 {'URL': 'https://brokers.interexo.com/united-states_florida/compa

In [122]:
# extract the text out of candidates "bios"
def extract_text_from_url(url, data, visited_urls):
    if url in visited_urls:
        print(f'url already in visited_urls: {url}')
        return None, None
    
    try:
        response = requests.get(url, headers=headers, allow_redirects=True, timeout=8)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.text
    except Exception as e:
        stdout = f'{idx:5d}  url: {url}  I had an exception {e}'
        print(stdout)
        visited_urls.add(url)
        data[url] = stdout
        return None, None
    
    data[url] = text
    visited_urls.add(url)
    print(f'{idx:5d} {len(data[url]):6d} chars  {url} ')
    return text

data = dict()
visited_urls = set()
for idx, bio in enumerate(bios):
    url = bio['URL']
    text = extract_text_from_url(url, data, visited_urls)
    num_chars = len(text)
    time.sleep(.1)


    0    793 chars  https://brokers.interexo.com/united-states_florida/company/gps-property-group/abel-jimenez-feb 
    1    699 chars  https://brokers.interexo.com/united-states_florida/company/gps-property-group 
    2   1444 chars  https://brokers.interexo.com/united-states/company/transworld-business-advisors/abel-bastida-64e 
    3   1613 chars  https://brokers.interexo.com/united-states_ohio/company/transworld-business-advisors 
    4   1441 chars  https://brokers.interexo.com/united-states/company/transworld-business-advisors/abigail-king-87e 
    5   2966 chars  https://brokers.interexo.com/united-states_florida/company/healthcare-merger-acquisitions/abigail-widynski-5c2 
    6    809 chars  https://brokers.interexo.com/united-states_georgia/company/healthcare-merger-acquisitions 
    7   1386 chars  https://brokers.interexo.com/united-states_texas/company/ideal-properties/abner-padora-5f2 
    8    693 chars  https://brokers.interexo.com/united-states_texas/company/ideal-prope

In [125]:
from openai import OpenAI
client = OpenAI()

results = []
start = time.time()
model = "gpt-4"


for idx, (url,  text) in enumerate(data.items()):
    s0 = time.time()
    text = data[url]

    prompt = """Here's some text I extracted from a webpage:\n""" + text +\
    """ \n Extract the following Named Entities from the above text: "First Name", "Middle Name", "Last Name", "Profession", """ +\
    """Company Name", "Phone Number", "Email", "Address" , "State", "Zipcode", "Bio", "Specialty Areas" """ +\
    """and  put them in a json file with the keys I specified. If the information is missing """ +\
    """leave the value of that key blank. """+\
    """Only return a JSON formatted text. Don't say anything else, don't write any python code. """ +\
    """If the given text contains more than one individual, create a separate json file and return a list of json files"""
    """If any of the values in the JSON file is a list of items, make sure you wrap the items in square brackets compatible with python syntax."""

    completion = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": "You are an assistant that extracts named entities from provided text, and returns an answer that's only comprised of " +\
         "json formatted text"},
        {"role": "user", "content": prompt}
      ]
    )

    content = completion.choices[0].message.dict()['content']
    if ",}" in content:
        print('==> replacing ",}", "}" ' + f" {print_runtime(start, False)}")
        content = content.replace(',}', '}')
    if ",\n" in content:
        print(f'==> replacing ",\n" with ","' + f" {print_runtime(start, False)}")
        content = content.replace(",\n", ",")

    json_output = json.loads(content)
    if type(json_output) == list:
        for item in json_output:
            item['URL'] = url
            results.append(item)
    else:
        json_output['URL'] = url
        results.append(json_output)
    print(f'idx:{idx} finished. len(results):{len(results)} {print_runtime(start, False)} \n')



==> replacing ",
" with "," (0 min  4 sec)
idx:0 finished. len(results):1 (0 min  4 sec) 

==> replacing ",
" with "," (0 min  8 sec)
idx:1 finished. len(results):2 (0 min  3 sec) 

==> replacing ",
" with "," (0 min 16 sec)
idx:2 finished. len(results):3 (0 min  8 sec) 

==> replacing ",
" with "," (0 min 25 sec)
idx:3 finished. len(results):4 (0 min  8 sec) 

==> replacing ",
" with "," (0 min 34 sec)
idx:4 finished. len(results):5 (0 min  9 sec) 

==> replacing ",
" with "," (0 min 57 sec)
idx:5 finished. len(results):6 (0 min 22 sec) 

==> replacing ",
" with "," (1 min  1 sec)
idx:6 finished. len(results):7 (0 min  3 sec) 



KeyboardInterrupt: 

In [126]:
content

'[{\n"First Name": "","Middle Name": "","Last Name": "","Profession": "","Company Name": "Healthcare Merger & Acquisitions","Phone Number": "","Email": "","Address": "","State": "GA","Zipcode": "","Bio": "Healthcare Merger & Acquisitions is located in GA and offers the professional services that successfully bring buyers and sellers together.","Specialty Areas": ""\n}]'

In [134]:
np.exp(-0.031678528)

0.9688179798773064

In [127]:
import pandas as pd
df = pd.DataFrame(results)
df.to_csv('named_entities.csv')
df

,First Name,Middle Name,Last Name,Profession,Company Name,Phone Number,Email,Address,State,Zipcode,Bio,Specialty Areas,URL
0,Abel,,Jimenez,Business Broker,Gps Property Group,407-452-4119,,,FL,,Abel Jimenez is a business broker of the well...,,https://brokers.interexo.com/united-states_flo...
1,,,,,"Gps Property Group, Interexo",,,,FL,,Gps Property Group is located in FL and offers...,,https://brokers.interexo.com/united-states_flo...
2,Abel,,Bastida,Broker,Transworld Business Advisors,"[262-287-3478, 800-205-8244]",abastida@tworld.com,,,,"As an agent with Transworld Business Advisors,...",,https://brokers.interexo.com/united-states/com...
3,,,,,Transworld Business Advisors - Interexo Busine...,800-205-7753,,"4786 Dressler Rd Nw, Suite 195",OH,44718,Transworld Business Advisors is located in OH ...,,https://brokers.interexo.com/united-states_ohi...
4,Abigail,,King,Broker,Transworld Business Advisors,"[770-225-2910, 800-205-7808]",aking@tworld.com,,United States,,"As an agent with Transworld Business Advisors,...",,https://brokers.interexo.com/united-states/com...
5,Abigail,S.,Widynski,Business Broker,"Healthcare Merger & Acquisitions, Interexo Bus...",800-201-3143,abigail@5thaavc.com,,FL,,Abigail Widynski is a business broker of the w...,Healthcare Merger & Acquisitions,https://brokers.interexo.com/united-states_flo...
6,,,,,Healthcare Merger & Acquisitions,,,,GA,,Healthcare Merger & Acquisitions is located in...,,https://brokers.interexo.com/united-states_geo...


In [128]:
results[-1]

{'First Name': '',
 'Middle Name': '',
 'Last Name': '',
 'Profession': '',
 'Company Name': 'Healthcare Merger & Acquisitions',
 'Phone Number': '',
 'Email': '',
 'Address': '',
 'State': 'GA',
 'Zipcode': '',
 'Bio': 'Healthcare Merger & Acquisitions is located in GA and offers the professional services that successfully bring buyers and sellers together.',
 'Specialty Areas': '',
 'URL': 'https://brokers.interexo.com/united-states_georgia/company/healthcare-merger-acquisitions'}